# Data Understanding & Data Preparation

### Import Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from sklearn.feature_selection import SelectKBest

client = 'TOOL_CLIENT.csv'
sales  = 'TOOL_SALES.csv'
joint = 'TOOL_JOINT.csv'

In [2]:
df_client = pd.read_csv(client)
df_sales = pd.read_csv(sales)

df = pd.merge(df_client, df_sales, on='CLIENT_ID')

# df = pd.read_csv(joint)

# lower case and replace spaces with underscores in column names
original_columns = df.columns
renamed_columns = [col.lower().replace(" ", "_") for col in original_columns]
column_mapping = dict(zip(original_columns, renamed_columns))
df.rename(columns=column_mapping, inplace=True)

df.head()

/tmp/ipykernel_165607/751546178.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv(sales)


,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,unit,family_code,group_code,cancelled
0,9306,2005-11-15 00:00:00,BZ,11000,6,8659.81,D,3d,201709,12585,0,C,937.94,P,XBXV2EB,XBXV2EB0102,NaN
1,9306,2005-11-15 00:00:00,BZ,11000,6,8659.81,D,3d,201709,13431,0,C,641.09,P,XBXV2AB,XBXV2AB0102,NaN
2,9306,2005-11-15 00:00:00,BZ,11000,6,8659.81,D,3d,201709,1380,0,C,0.00,P,XBXV2EB,XBXV2EB0102,NaN
3,9306,2005-11-15 00:00:00,BZ,11000,6,8659.81,D,3d,202010,3493,1,C,1341.14,P,XBXV1AF,XBXV1AF0201,NaN
4,939,2005-11-15 00:00:00,LE,15500,2,681.26,E,3d,201705,8241,0,B,64.51,P,XAES4AA,XAES4AA0101,X


In [ ]:
df.info()

In [3]:
# converting date columns to datetime
df['client_create_date'] = pd.to_datetime(df['client_create_date'])
df['yyyymm'] = pd.to_datetime(df['yyyymm'].astype(str), format='%Y%m')
df['client_id'] = df['client_id'].astype(str) # 
df['cancelled'] = df['cancelled'] == 'X'
df['unit'] = df['unit'] == 'P'

In [ ]:
df.info()

In [ ]:
# Create a copy of the data
df_total = df.copy()

In [ ]:
# Initial size of the data
df.shape

In [ ]:
# exporting the merged data

# df.to_csv('TOOL_JOINT.csv', index=False)

Create a new variable to identify client

Create a new variable to identify unique sales

In [4]:
# creating a new column for identifie the unique sales
df['sales_id'] = df['client_id'].astype(str) + '_' + df['yyyymm'].dt.strftime('%Y%m')

df.head()

,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,unit,family_code,group_code,cancelled,sales_id
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,True,XBXV2EB,XBXV2EB0102,False,9306_201709
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,True,XBXV2AB,XBXV2AB0102,False,9306_201709
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,True,XBXV2EB,XBXV2EB0102,False,9306_201709
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,True,XBXV1AF,XBXV1AF0201,False,9306_202010
4,939,2005-11-15,LE,15500,2,681.26,E,3d,2017-05-01,8241,0,B,64.51,True,XAES4AA,XAES4AA0101,True,939_201705


In [5]:
# creating new dataframes with no canceled orders
no_canceled = df[df['cancelled'] == False]
print('Number of clients with no canceled orders:', no_canceled.shape)
print('Porcentage of no canceled/total:', (no_canceled.shape[0]/df.shape[0])*100)
# no_canceled.head()

Number of clients with no canceled orders: (1947739, 18)
Porcentage of no canceled/total: 94.99085322287948


In [6]:
# checking the number of unique sales per client
client_sales_count = no_canceled.groupby('client_id')['sales_id'].nunique().sort_values(ascending=False)
print(client_sales_count)

client_id
51708    60
19562    58
9659     58
27926    58
37126    57
         ..
39817     1
7320      1
73199     1
56057     1
1         1
Name: sales_id, Length: 85714, dtype: int64


In [ ]:
client_sales_count.describe()

In [7]:
# filtering the clients with more than one purchase - reactivation
client_sales_count.drop(client_sales_count[client_sales_count < 2].index, inplace=True)

pluri_client = no_canceled[no_canceled['client_id'].isin(client_sales_count.index)]
pluri_client['n_purchases'] = pluri_client.groupby('client_id')['sales_id'].transform('nunique')

print(pluri_client.shape)
print(pluri_client.info())

(1840920, 19)
<class 'pandas.core.frame.DataFrame'>
Index: 1840920 entries, 0 to 2048901
Data columns (total 19 columns):
 #   Column              Dtype         
---  ------              -----         
 0   client_id           object        
 1   client_create_date  datetime64[ns]
 2   region              object        
 3   trade_sector        int64         
 4   n_employees         int64         
 5   economic_pot        float64       
 6   eco_pot_class       object        
 7   risk_cat            object        
 8   yyyymm              datetime64[ns]
 9   item_id             int64         
 10  flg_tool            int64         
 11  sales_channel       object        
 12  net                 float64       
 13  unit                bool          
 14  family_code         object        
 15  group_code          object        
 16  cancelled           bool          
 17  sales_id            object        
 18  n_purchases         int64         
dtypes: bool(2), datetime64[ns](2), fl

/tmp/ipykernel_165607/2084113973.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluri_client['n_purchases'] = pluri_client.groupby('client_id')['sales_id'].transform('nunique')


In [ ]:
pluri_client.head()

In [ ]:
# Flag Tool
pluri_client['risk_cat'].value_counts()

In [ ]:
for c in pluri_client.columns:
    print(c)
    print(pluri_client[c].value_counts())

We have now a new dataset cointaining only client with more than one purchase

In [ ]:
plt.figure(figsize=(20, 6))
sns.countplot(x='n_purchases', data=pluri_client, hue='unit')
plt.xlabel('Number of Purchases')
plt.ylabel('Number of Clients')
plt.title('Number of Purchases per Client')
plt.show()

On this point I'd like asking business about articles that aren't "Unit". I suppose that could be substitute parts for other articles or similar, which a client buys because it absolutly needs it and a marketing campaign about is unuseful. So I decide to drop rows containing non-piece units

In [8]:
# dropping unnecessary columns
pluri_client.drop(pluri_client[pluri_client['unit'] == False].index, inplace=True)
pluri_client.drop('unit', axis=1, inplace=True)
pluri_client.drop('cancelled', axis=1, inplace=True)

pluri_client.head()

/tmp/ipykernel_165607/4086462661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluri_client.drop(pluri_client[pluri_client['unit'] == False].index, inplace=True)
/tmp/ipykernel_165607/4086462661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluri_client.drop('unit', axis=1, inplace=True)
/tmp/ipykernel_165607/4086462661.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluri_client.drop('cancelled', axis=1, inplace=True)


,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,family_code,group_code,sales_id,n_purchases
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,XBXV2EB,XBXV2EB0102,9306_201709,2
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,XBXV2AB,XBXV2AB0102,9306_201709,2
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,XBXV2EB,XBXV2EB0102,9306_201709,2
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,XBXV1AF,XBXV1AF0201,9306_202010,2
7,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,0,B,20.02,XBSI4AA,XBSI4AA0101,8321_201804,7


In [ ]:
pluri_client.shape

### Next steps: 
- Days since last purchase for same client
- Analyze time between purchases

- Create a DF with total amount for each sales_id

In [9]:
# extracting new variables
sales_net = pluri_client.groupby('sales_id')['net'].sum()   # total sales per sale
sales_n_purchases = pluri_client.groupby('sales_id')['n_purchases'].max()   # number of purchases per sale

sales_time = pluri_client[['sales_id', 'client_id', 'yyyymm']].drop_duplicates().set_index('client_id')   # time of the sale
sales_time = sales_time.sort_values(by=['client_id', 'yyyymm'])

sales_time['time_diff'] = sales_time.groupby('client_id')['yyyymm'].diff().dt.days / 365.25 # time between sales in years
sales_id_time_diff = sales_time[['sales_id', 'time_diff']].set_index('sales_id')    # time between sales per sale (to easly merge with the main dataframe)

In [10]:
# adding the new variables to the main dataframe
pluri_client['sales_net'] = pluri_client['sales_id'].map(sales_net)
pluri_client['time_diff'] = pluri_client['sales_id'].map(sales_id_time_diff['time_diff'].to_dict())
pluri_client.head()

/tmp/ipykernel_165607/278983237.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluri_client['sales_net'] = pluri_client['sales_id'].map(sales_net)
/tmp/ipykernel_165607/278983237.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluri_client['time_diff'] = pluri_client['sales_id'].map(sales_id_time_diff['time_diff'].to_dict())


,client_id,client_create_date,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,yyyymm,item_id,flg_tool,sales_channel,net,family_code,group_code,sales_id,n_purchases,sales_net,time_diff
0,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,12585,0,C,937.94,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN
1,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,13431,0,C,641.09,XBXV2AB,XBXV2AB0102,9306_201709,2,1579.03,NaN
2,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2017-09-01,1380,0,C,0.00,XBXV2EB,XBXV2EB0102,9306_201709,2,1579.03,NaN
3,9306,2005-11-15,BZ,11000,6,8659.81,D,3d,2020-10-01,3493,1,C,1341.14,XBXV1AF,XBXV1AF0201,9306_202010,2,1341.14,3.08282
7,8321,2005-11-15,LE,15500,2,681.26,E,T8,2018-04-01,6776,0,B,20.02,XBSI4AA,XBSI4AA0101,8321_201804,7,20.02,NaN


In [ ]:
sales_time.info()

In [11]:
# creating a new dataframe for sales
sales_net = sales_net.reset_index()
sales_n_purchases = sales_n_purchases.reset_index()
sales_time.reset_index(inplace=True)

merged = pd.merge(sales_time, sales_net, on='sales_id')
sales = pd.merge(merged, sales_n_purchases, on='sales_id')

df_original = df
df = pluri_client

In [12]:
# adding the new variables to the sales dataframe
sales_columns = ['region', 'trade_sector', 'n_employees', 'economic_pot',
                 'eco_pot_class', 'risk_cat', 'flg_tool', 'sales_channel', 'sales_id']  # columns to be added to the sales dataframe

for c in sales_columns: # adding the columns to the sales dataframe
    col = df.groupby('sales_id')[c].first()
    sales = pd.merge(sales, col, left_on='sales_id', right_index=True)

sales.drop(['sales_id_x', 'sales_id_y'], axis=1, inplace=True)

In [ ]:
sales.head(5)

In [ ]:
sales['sales_channel'].value_counts()['D']

In [ ]:
# Filter clients with at least one gap ≥ 2 years
clients_with_gaps = sales[sales['time_diff'] >= 2]['client_id'].unique()

# Count clients with and without gaps
total_clients = sales['client_id'].nunique()
clients_with_gaps_count = len(clients_with_gaps)
clients_without_gaps_count = total_clients - clients_with_gaps_count

# Prepare data for the pie chart
labels = ['With gap ≥ 2 years', 'Without gap ≥ 2 years']
sizes = [clients_with_gaps_count, clients_without_gaps_count]
colors = ['#ff9999', '#66b3ff']

# Plot the pie chart
plt.figure(figsize=(6,6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90, shadow=True)
plt.title('Proportion of customers with gap ≥ 2 years of each purchase')
plt.show()

P.S. de-comment the follow cell to save files

In [ ]:
# sales.to_csv('sales.csv', index=False)
# pluri_client.to_csv('TOOLS_PREP.csv', index=False)

Now we have 2 DataFrame to use for our analysis
- df: the complete DataFrame with all the variables
- sales: the DataFrame with the details of sales

## Working on association rules

In [ ]:
# one-hot encoding the itemsets for the apriori algorithm
itemsets_series = df.groupby('client_id')['item_id'].apply(list)
encoder = TransactionEncoder()
itemsets = itemsets_series.tolist()

onehot = encoder.fit_transform(itemsets, sparse=True)
onehot = pd.DataFrame.sparse.from_spmatrix(onehot, columns = encoder.columns_)
onehot.columns = [str(col) for col in onehot.columns]
onehot.info()

frequent_itemsets = apriori(onehot, min_support=0.01, max_len = 4, use_colnames=True, low_memory=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)
print(len(rules))

In [ ]:
# filtering and ordering the rules
rules.sort_values(by=['support', 'confidence'], ascending=False, inplace=True)
print('We have a total of {} rules'.format(rules.shape[0]))
rules.head(20)

## Model

In [ ]:
sales.head()

In [13]:
# fill NaN on sales - time_diff
sales_copy = sales.copy()
sales_copy['time_diff'] = sales_copy['time_diff'].fillna(0)

# sales_10ktime = sales_copy.copy()
# sales_copy['time_diff'] = sales_copy['time_diff'].fillna(sales_copy['time_diff'].max() + 5)
sales_copy.head()

,sales_id,client_id,yyyymm,time_diff,net,n_purchases,region,trade_sector,n_employees,economic_pot,eco_pot_class,risk_cat,flg_tool,sales_channel
0,10_201904,10,2019-04-01,0.000000,23.25,2,PN,21100,1,7000.00,D,5a,0,B
1,10_202103,10,2021-03-01,1.916496,294.70,2,PN,21100,1,7000.00,D,5a,0,B
2,100_201702,100,2017-02-01,0.000000,148.01,5,PT,11100,2,2345.86,E,3a,0,B
3,100_201804,100,2018-04-01,1.160849,531.73,5,PT,11100,2,2345.86,E,3a,0,B
4,100_201904,100,2019-04-01,0.999316,113.64,5,PT,11100,2,2345.86,E,3a,0,B


In [14]:
backup_sales = sales_copy.copy()

In [17]:
def create_target(element, clients):
    # print(f"Element: {type(element)}, clients={type(clients[0])}")
    return 1 if int(element) in clients else 0


# Step 1: Identify clients with at least one gap ≥ 2 years
clients_with_gaps = sales_copy[sales_copy['time_diff'] >= 2]['client_id'].to_list()
clients_with_gaps = [int(c_id) for c_id in clients_with_gaps]

# Step 2: Create target column
sales_copy['target'] = sales_copy['client_id'].apply(lambda x: create_target(x, clients_with_gaps))

In [19]:
sales_copy['client_id'] = sales_copy['client_id'].astype(str)
clients_with_gaps = set(map(str, clients_with_gaps))  # Convert all to string

In [20]:
sales_copy['target'].value_counts()

target
0    369836
1     39684
Name: count, dtype: int64

In [ ]:
# Step 3: Check results
sales_copy[['client_id', 'time_diff', 'target']].head(20)

In [21]:
from sklearn.feature_selection import SelectKBest, f_classif

# Select numerical columns for SelectKBest
numerical_features = sales_copy.select_dtypes(include=['int64', 'float64']).columns.drop('target')

sales_copy['yyyymm'] = sales_copy['yyyymm'].astype(str)

# Apply SelectKBest
X = sales_copy[numerical_features]
y = sales_copy['target']

selector = SelectKBest(score_func=f_classif, k='all')  # Select all features for ranking
X_selected = selector.fit(X, y)

# Show feature scores
feature_scores = pd.DataFrame({'Feature': numerical_features, 'Score': X_selected.scores_})
print(feature_scores.sort_values(by='Score', ascending=False))

        Feature         Score
0     time_diff  48850.325976
2   n_purchases  34347.592242
5  economic_pot   4051.416293
6      flg_tool    741.107058
3  trade_sector    178.266110
4   n_employees      4.409403
1           net      0.034233


In [ ]:
sales_copy[numerical_features].nunique()


Test with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# Get feature importances
feature_importance = pd.DataFrame({'Feature': numerical_features, 'Importance': rf.feature_importances_})
print(feature_importance.sort_values(by='Importance', ascending=False))